# Components

### Import Libraries

In [26]:
from qiskit import QuantumCircuit as qc, QuantumRegister as qr
from qiskit.circuit.gate import Gate
from qiskit.quantum_info import Statevector

import numpy as np
from functools import cache

### Define Variables

In [27]:
n_bits = 5

circuitDarkMode = {
    "backgroundcolor": "#222222",
    "linecolor": "#FFFFFF",
    "textcolor": "#FFFFFF",
    "gatetextcolor": "#FFFFFF",
}

### Addition Circuit

In [28]:
def rshiftGate(inReg: qr, outReg: qr, rshift: int, n:int) -> Gate:
    circuit = qc(inReg, outReg, name=f"{rshift=}")

    for i in range(n-rshift):
        circuit.cx(inReg[i+rshift], outReg[i])
    for i in range(n-rshift, n):
        circuit.cx(inReg[n-1], outReg[i])

    return circuit.to_gate()

def shiftAdditionGate(
        xReg: qr, yReg: qr, rsReg: qr, rshift: int = 0
    ) -> Gate:
    """Adds right shifted y to x register. Respects two's complement sign stuff.
        Based on the simplest version of an algorithm from:
            Quantum Addition Circuits and Unbounded Fan-Out
            Authors: Yasuhiro Takahashi, Seiichiro Tani, Noboru Kunihiro

    Args:
        xReg   (qr): Register to be added to
        yReg   (qr): Register which has input
        rsReg  (qr): Register to hold the bitshifted version of y.
        rshift (int, optional): The amount right shifted, non-negative only. Defaults to 0.

    Returns:
        Gate: Gate for the desired transformation
    """
    n       = min(len(xReg), len(yReg), len(rsReg))
    circuit = qc(xReg, yReg, rsReg, name=f"+y>>{rshift}")

    #Prepare shifted y
    circuit.append(
        rshiftGate(inReg=yReg, outReg=rsReg, rshift=rshift, n=n),
        yReg[:] + rsReg[:]
    )

    #Step 1
    for i in range(1,n):
        circuit.cx(rsReg[i], xReg[i])
    
    #Step 2
    for i in range(n-2, 0, -1):
        circuit.cx(rsReg[i], rsReg[i+1])

    #Step 3
    for i in range(n-1):
        circuit.ccx(rsReg[i], xReg[i], rsReg[i+1])

    #Step 4
    for i in range(n-1, 0, -1):
        circuit.cx(rsReg[i], xReg[i])
        circuit.ccx(rsReg[i-1], xReg[i-1], rsReg[i])

    #Step 5
    for i in range(1, n-1):
        circuit.cx(rsReg[i], rsReg[i+1])
    
    #Step 6
    for i in range(n):
        circuit.cx(rsReg[i], xReg[i])

    #Uncompute shifted y
    circuit.append(
        rshiftGate(inReg=yReg, outReg=rsReg, rshift=rshift, n=n).inverse(),
        yReg[:] + rsReg[:]
    )

    # return circuit #temp
    return circuit.to_gate()

# rshiftGate(qr(n_bits, name="in"), qr(n_bits, name="out"), 2, n_bits).draw('mpl', fold=-1, style=circuitDarkMode)

# shiftAdditionGate(
#     xReg=qr(n_bits,name="x"), yReg=qr(n_bits,name="y"), rsReg=qr(n_bits,name="s"), rshift=0
# ).draw('mpl', fold=-1, style=circuitDarkMode)

### Testing the rShift Addition Circuit

In [29]:
def intToBits(value: int, n: int=0) -> str:
    return f"{value:0>{n}b}"[::-1] 

def bitsToInt(value: str) -> int:
    return int(value[::-1], 2)

def intListToBits(values: list[int], n: int) -> str:
    string = []
    for v in values:
        string.append(f"{v:0>{n}b}"[::-1])

    return ''.join(string)[::-1]

def bitsToIntList(string: str, n: int) -> list[int]:
    string = string[::-1]
    values = []

    for i in range(len(string)//n):
        values.append(int(string[i*n:(i+1)*n][::-1], 2))
    
    return values

def stateToStr(value: Statevector) -> str:
    return value.draw('latex_source').replace(r'\rangle', '>')

def stateToIntList(value: Statevector, n: int) -> list[int]:
    string = "".join(filter(str.isdigit, stateToStr(value)))
    return bitsToIntList(string, n)


xReg   = qr(n_bits,name="x")
yReg   = qr(n_bits,name="y")
auxReg = qr(n_bits,name="s")

x      = 5
y      = 8
rshift = 3

inState = intListToBits([x,y,0], n_bits)
# print(inState)

addCircuit = qc(xReg, yReg, auxReg)
addCircuit.append(
    shiftAdditionGate(xReg, yReg, auxReg, rshift),
    xReg[:] + yReg[:] + auxReg[:]
)

# addCircuit.decompose(reps=2).draw('mpl', style=circuitDarkMode)
# print(addCircuit.decompose().draw())

state = Statevector.from_label(inState)
print(f"in:  {stateToIntList(state, n_bits)}")
print(f"    {stateToStr(state)}")
state = state.evolve(addCircuit)
print(f"out: {stateToIntList(state, n_bits)}")
print(f"    {stateToStr(state)}")


in:  [5, 8, 0]
     |000000100000101>
out: [6, 8, 0]
     |000000100000110>


### Mult Circuit

In [31]:

@cache
def fib(i:int) -> int: 
    '''
    Fibonacci numbers { 1,1,3,5,8,13, ... }

    i (int): Index of Fibonacci number

    return (int): i^th Fibonacci number
    '''
    return int(np.round(
        1/(5**.5) * (((1+5**.5)/2)**(i+1)-((1-5**.5)/2)**(i+1))
    ))


def multGate(
        xReg: qr, aux1Reg: qr, aux2Reg: qr, rsReg: qr, n: int, m: int
    ) -> Gate:
    '''
    In place multiplication integer by 1+2^(-m) with some error depending on 
    how clean the auxiliary registers are.
    Note: the aux_2 register can gain some error

    xReg     (qr): Register being multiplied
    aux1Reg  (qr): First auxiliary register, used to help clean aux_2
    aux2Reg  (qr): Second auxiliary register, stores a copy of xReg
    rsReg    (qr): Auxiliary Register for the addition stuff
    n       (int): Number of bits representing xReg
    m       (int): xReg is multiplied by 1+2^(-m)

    Return:
        Gate: Circuit to perform the operation
    '''
    circuit = qc(xReg, aux1Reg, aux2Reg, rsReg, name=f"x(1+2^(-m))")

    circuit.append(
        shiftAdditionGate(aux2Reg, xReg, rsReg, rshift=0),
        aux2Reg[:] + xReg[:] + rsReg[:]
    )
    circuit.append(
        shiftAdditionGate(xReg, aux2Reg, rsReg, rshift=m),
        xReg[:] + aux2Reg[:] + rsReg[:]
    )
    circuit.append(
        shiftAdditionGate(aux1Reg, xReg, rsReg, rshift=0),
        aux1Reg[:] + xReg[:] + rsReg[:]
    )
    # aux_2, xReg = add(aux_2, xReg, 0, n=n)
    # x, aux_2 = add(x, aux_2, m, n=n)
    # aux_1, x = add(aux_1, x, 0, n=n)

    # inState = intListToBits([5,0,0,0], n_bits) #temp
    # state = Statevector.from_label(inState) #temp
    
    for i in range(int(1+2*np.ceil(np.log(n/m)/np.log((1+5**.5))))):
        # print(stateToIntList(state.evolve(circuit), n)) #temp
        negative = fib(i)%2==1
        if i%2 == 0:
            circuit.append(
                shiftAdditionGate(xReg, aux1Reg, rsReg, m*fib(i)) if not negative
                else shiftAdditionGate(xReg, aux1Reg, rsReg, m*fib(i)).inverse(),
                xReg[:] + aux1Reg[:] + rsReg[:]
            )
            # x, aux_1 = add(x, aux_1, m*fib(i), n, negativeY=(fib(i)%2==1))
        else:
            circuit.append(
                shiftAdditionGate(aux1Reg, xReg, rsReg, m*fib(i)) if not negative
                else shiftAdditionGate(aux1Reg, xReg, rsReg, m*fib(i)).inverse(),
                aux1Reg[:] + xReg[:] + rsReg[:]
            )
            # aux_1, x = add(aux_1, x, m*fib(i), n, negativeY=(fib(i)%2==1))
            
        
    circuit.append(
        shiftAdditionGate(aux2Reg, aux1Reg, rsReg, 0).inverse(),
        aux2Reg[:] + aux1Reg[:] + rsReg[:]
    )
    # aux_2, aux_1 = add(aux_2, aux_1, 0, n=n, negativeY=True)
    
    for i in range(int(1+2*np.ceil(np.log(n/m)/np.log((1+5**.5))))-1, -1, -1):
        negative = fib(i)%2==1
        if i%2 == 0:
            circuit.append(
                shiftAdditionGate(xReg, aux1Reg, rsReg, m*fib(i)) if negative
                else shiftAdditionGate(xReg, aux1Reg, rsReg, m*fib(i)).inverse(),
                xReg[:] + aux1Reg[:] + rsReg[:]
            )
            # x, aux_1 = add(x, aux_1, m*fib(i), n, negativeY=not (fib(i)%2==1))
        else:
            circuit.append(
                shiftAdditionGate(aux1Reg, xReg, rsReg, m*fib(i)) if negative
                else shiftAdditionGate(aux1Reg, xReg, rsReg, m*fib(i)).inverse(),
                aux1Reg[:] + xReg[:] + rsReg[:]
            )
            # aux_1, x = add(aux_1, x, m*fib(i), n, negativeY=not (fib(i)%2==1))

    circuit.append(
        shiftAdditionGate(aux1Reg, xReg, rsReg, 0).inverse(),
        aux1Reg[:] + xReg[:] + rsReg[:]
    )
    # aux_1, x = add(aux_1, x, 0, n, negativeY=True)

    return circuit.to_gate()

xReg    = qr(n_bits, name="x");     aux1Reg = qr(n_bits, name="aux_1")
aux2Reg = qr(n_bits, name="aux_2"); rsReg   = qr(n_bits, name="rs")

mult = qc(xReg, aux1Reg, aux2Reg, rsReg)

mult.append(
    multGate(xReg, aux1Reg, aux2Reg, rsReg, n=n_bits, m=1),
    xReg[:] + aux1Reg[:] + aux2Reg[:] + rsReg[:]
)

# mult.decompose().draw('mpl', fold=-1, style=circuitDarkMode)
# print(mult.draw())

inState = intListToBits([5,0,0,0], n_bits)
state = Statevector.from_label(inState)
print(f"in:  {stateToIntList(state, n_bits)}")
print(f"    {stateToStr(state)}")
state = state.evolve(mult)
print(f"out: {stateToIntList(state, n_bits)}")
print(f"    {stateToStr(state)}")


[7, 7, 5, 0]
[4, 7, 5, 0]
[4, 5, 5, 0]
[5, 5, 5, 0]
[5, 5, 5, 0]
in:  [5, 0, 0, 0]
     |00000000000000000101>
out: [7, 0, 0, 0]
     |00000000000000000111>
